In [ ]:
!pip install scikeras

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.impute import KNNImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import numpy as np

# Load the data
data = pd.read_csv('TSS_data.csv')

# Impute NaN values for the entire dataset using KNN
imputer = KNNImputer(n_neighbors=5)
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# Separate features and target variables
X = data_imputed.iloc[:, :-3]  # Exclude the last three columns (additional targets)
y_tss = data_imputed['TSS']
y_chla = data_imputed['Chla']
y_cdom = data_imputed['CDOM']

# Standardize the features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

def create_mdn_model(units=64, num_components=3, learning_rate=0.001):
    model = Sequential([
        Dense(units, activation='relu', input_dim=X_scaled.shape[1]),
        Dense(num_components * 3, activation='linear')  # 3 parameters per component
    ])

    def mdn_loss(y_true, y_pred):
        # Extract mean, log_sigma, and alpha using indexing
        num_params = num_components * 3
        mean = y_pred[:, :num_components]
        log_sigma = y_pred[:, num_components:2*num_components]
        alpha = y_pred[:, 2*num_components:]

        sigma = K.exp(log_sigma)

        # Gaussian probability density function
        pdf = K.exp(-0.5 * K.square((y_true - mean) / (sigma + 1e-8))) / (sigma + 1e-8) / np.sqrt(2 * np.pi)

        # Mixture of Gaussians
        loss = -K.log(K.sum(alpha * pdf, axis=1, keepdims=True) + 1e-8)
        return loss

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=mdn_loss)
    return model

# Perform 10-fold cross-validation with different hyperparameter combinations for TSS
kf = KFold(n_splits=10, shuffle=True, random_state=42)

param_combinations = [(32, 2, 0.001), (64, 2, 0.001), (128, 2, 0.001),
                      (32, 3, 0.001), (64, 3, 0.001), (128, 3, 0.001),
                      (32, 4, 0.001), (64, 4, 0.001), (128, 4, 0.001)]

def cross_validate_and_save(X, y, target_name, param_combinations):
    mape_scores = []

    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]

        for units, num_components, learning_rate in param_combinations:
            mdn_model = create_mdn_model(units=units, num_components=num_components, learning_rate=learning_rate)

            # Implement early stopping
            early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

            mdn_model.fit(
                X_train, y_train,
                epochs=100,
                batch_size=32,
                validation_data=(X_test, y_test),
                callbacks=[early_stopping],
                verbose=0
            )

            y_pred = mdn_model.predict(X_test)

            mape = mean_absolute_percentage_error(y_test, y_pred[:, :1].flatten())  # Use mean for prediction
            mape_scores.append((units, num_components, learning_rate, mape))

            # Save predicted values
            predictions_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred[:, :1].flatten()})
            predictions_df.to_csv(f'{target_name}_predictions_fold_{fold}.csv', index=False)

    # Find the best hyperparameters based on the lowest average MAPE
    best_hyperparameters = min(mape_scores, key=lambda x: x[3])
    best_units, best_num_components, best_learning_rate, best_mape = best_hyperparameters

    # Print the best hyperparameters and average MAPE
    print(f'Best Hyperparameters for {target_name}: Units={best_units}, Num Components={best_num_components}, Learning Rate={best_learning_rate}')
    print(f'Average MAPE for {target_name} across 10 folds: {best_mape}')

    return best_hyperparameters, best_mape

# Cross-validate and save models for each target variable
best_params_tss, mape_tss = cross_validate_and_save(X_scaled, y_tss, 'TSS', param_combinations)
best_params_chla, mape_chla = cross_validate_and_save(X_scaled, y_chla, 'Chla', param_combinations)
best_params_cdom, mape_cdom = cross_validate_and_save(X_scaled, y_cdom, 'CDOM', param_combinations)


16/16 [==============================] - 0s 2ms/step
Best Hyperparameters for TSS: Units=128, Num Components=4, Learning Rate=0.001
Average MAPE for TSS across 10 folds: 0.8678050003305394
16/16 [==============================] - 0s 2ms/step
Best Hyperparameters for Chla: Units=64, Num Components=3, Learning Rate=0.001
Average MAPE for Chla across 10 folds: 0.7004991303428644
16/16 [==============================] - 0s 2ms/step
Best Hyperparameters for CDOM: Units=64, Num Components=3, Learning Rate=0.001
Average MAPE for CDOM across 10 folds: 1.7807935947666316


In [ ]:
#tss_fine_tuning
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import numpy as np

# Function to save predictions to a CSV file
def save_predictions(predictions, target_name):
    predictions.reset_index(drop=True, inplace=True)  # Reset index
    predictions.to_csv(f'{target_name}_predictions.csv', index=False)

# Load the data
data = pd.read_csv('TSS1.csv')  # Change the filename to your actual CSV file

# Impute NaN values for the entire dataset using KNN
imputer = KNNImputer(n_neighbors=5)
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# Separate features and target variables
X = data_imputed.iloc[:, :-1]  # Exclude the last column (target variable)
y_tss = data_imputed['TSS']

# Standardize the features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Define the MDN loss function
num_components = 1
def mdn_loss(y_true, y_pred):
    # Extract mean, log_sigma, and alpha using indexing
    num_params = num_components * 3
    mean = y_pred[:, :num_components]
    log_sigma = y_pred[:, num_components:2*num_components]
    alpha = y_pred[:, 2*num_components:]

    sigma = K.exp(log_sigma)

    # Gaussian probability density function
    pdf = K.exp(-0.5 * K.square((y_true - mean) / (sigma + 1e-8))) / (sigma + 1e-8) / np.sqrt(2 * np.pi)

    # Mixture of Gaussians
    loss = -K.log(K.sum(alpha * pdf, axis=1, keepdims=True) + 1e-8)
    return loss

# Function to fine-tune the existing MDN model
def fine_tune_mdn_model(existing_model, X, y, target_name, learning_rate=0.001):
    # Implement early stopping
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

    # Compile the existing model with the same loss function and optimizer
    existing_model.compile(optimizer=Adam(learning_rate=learning_rate), loss=mdn_loss)

    # Fine-tune the model on the new data
    existing_model.fit(
        X, y,
        epochs=100,  # You can adjust the number of epochs
        batch_size=32,
        callbacks=[early_stopping],
        verbose=1
    )

    # Save the fine-tuned MDN model
    existing_model.save(f'{target_name}_fine_tuned_mdn_model.h5')

    # Print the hyperparameters
    print("Hyperparameters:")
    print(existing_model.get_config()['layers'][1]['config'])  # Assuming the first layer is Dense

    # Generate predictions on the fine-tuned model
    y_pred = existing_model.predict(X)

    # Calculate and print MAPE
    mape = mean_absolute_percentage_error(y, y_pred[:, :1].flatten())  # Use mean for prediction
    print(f'MAPE for {target_name} after fine-tuning: {mape}')

    # Save predictions to CSV
    predictions_df = pd.DataFrame({'Actual': y, 'Predicted': y_pred[:, :1].flatten()})
    save_predictions(predictions_df, target_name)

    return mape

# Load the existing MDN model
existing_mdn_model = load_model('TSS_mdn_model.h5', custom_objects={'mdn_loss': mdn_loss})

# Fine-tune the MDN model for TSS
mape_tss_fine_tuned = fine_tune_mdn_model(existing_mdn_model, X_scaled, y_tss, 'TSS')




Epoch 1/100
1/1 [==============================] - 1s 780ms/step - loss: nan
Epoch 2/100
1/1 [==============================] - 0s 11ms/step - loss: nan
Epoch 3/100
1/1 [==============================] - 0s 11ms/step - loss: nan
Epoch 4/100
1/1 [==============================] - 0s 10ms/step - loss: nan
Epoch 5/100
1/1 [==============================] - 0s 11ms/step - loss: nan
Epoch 6/100
1/1 [==============================] - 0s 12ms/step - loss: nan
Epoch 7/100
1/1 [==============================] - 0s 9ms/step - loss: nan
Epoch 8/100
1/1 [==============================] - 0s 12ms/step - loss: nan
Epoch 9/100
1/1 [==============================] - 0s 14ms/step - loss: nan
Epoch 10/100
1/1 [==============================] - 0s 13ms/step - loss: nan
Hyperparameters:
{'name': 'dense_876', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': (None, 8), 'units': 128, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': No

In [ ]:
#chla_fine_tuning
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import numpy as np

# Function to save predictions to a CSV file
def save_predictions(predictions, target_name):
    predictions.reset_index(drop=True, inplace=True)  # Reset index
    predictions.to_csv(f'{target_name}_predictions.csv', index=False)

# Load the data
data = pd.read_csv('Chl1.csv')  # Change the filename to your actual CSV file

# Impute NaN values for the entire dataset using KNN
imputer = KNNImputer(n_neighbors=5)
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# Separate features and target variables
X = data_imputed.iloc[:, :-1]  # Exclude the last column (target variable)
y_chla = data_imputed['Chla']

# Standardize the features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Define the MDN loss function
num_components = 1
def mdn_loss(y_true, y_pred):
    # Extract mean, log_sigma, and alpha using indexing
    num_params = num_components * 3
    mean = y_pred[:, :num_components]
    log_sigma = y_pred[:, num_components:2*num_components]
    alpha = y_pred[:, 2*num_components:]

    sigma = K.exp(log_sigma)

    # Gaussian probability density function
    pdf = K.exp(-0.5 * K.square((y_true - mean) / (sigma + 1e-8))) / (sigma + 1e-8) / np.sqrt(2 * np.pi)

    # Mixture of Gaussians
    loss = -K.log(K.sum(alpha * pdf, axis=1, keepdims=True) + 1e-8)
    return loss

# Function to fine-tune the existing MDN model
def fine_tune_mdn_model(existing_model, X, y, target_name, learning_rate=0.001):
    # Implement early stopping
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

    # Compile the existing model with the same loss function and optimizer
    existing_model.compile(optimizer=Adam(learning_rate=learning_rate), loss=mdn_loss)

    # Fine-tune the model on the new data
    existing_model.fit(
        X, y,
        epochs=100,  # You can adjust the number of epochs
        batch_size=32,
        callbacks=[early_stopping],
        verbose=1
    )

    # Save the fine-tuned MDN model
    existing_model.save(f'{target_name}_fine_tuned_mdn_model.h5')

    # Print the hyperparameters
    print("Hyperparameters:")
    print(existing_model.get_config()['layers'][1]['config'])  # Assuming the first layer is Dense

    # Generate predictions on the fine-tuned model
    y_pred = existing_model.predict(X)

    # Calculate and print MAPE
    mape = mean_absolute_percentage_error(y, y_pred[:, :1].flatten())  # Use mean for prediction
    print(f'MAPE for {target_name} after fine-tuning: {mape}')

    # Save predictions to CSV
    predictions_df = pd.DataFrame({'Actual': y, 'Predicted': y_pred[:, :1].flatten()})
    save_predictions(predictions_df, target_name)

    return mape

# Load the existing MDN model
existing_mdn_model = load_model('Chla_mdn_model.h5', custom_objects={'mdn_loss': mdn_loss})

# Fine-tune the MDN model for Chla
mape_chla_fine_tuned = fine_tune_mdn_model(existing_mdn_model, X_scaled, y_chla, 'Chla')


Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 16.9505
Epoch 2/100
1/1 [==============================] - 0s 15ms/step - loss: 16.8543
Epoch 3/100
1/1 [==============================] - 0s 21ms/step - loss: 16.7701
Epoch 4/100
1/1 [==============================] - 0s 22ms/step - loss: 16.6966
Epoch 5/100
1/1 [==============================] - 0s 16ms/step - loss: 16.6325
Epoch 6/100
1/1 [==============================] - 0s 21ms/step - loss: 16.5768
Epoch 7/100
1/1 [==============================] - 0s 26ms/step - loss: 16.5283
Epoch 8/100
1/1 [==============================] - 0s 18ms/step - loss: 16.4871
Epoch 9/100
1/1 [==============================] - 0s 20ms/step - loss: 16.4512
Epoch 10/100
1/1 [==============================] - 0s 13ms/step - loss: 16.4199
Epoch 11/100
1/1 [==============================] - 0s 16ms/step - loss: 16.3925
Epoch 12/100
1/1 [==============================] - 0s 15ms/step - loss: 16.3683
Epoch 13/100
1/1 [=====================

In [ ]:
#cdom_fine_tuning
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import numpy as np

# Function to save predictions to a CSV file
def save_predictions(predictions, target_name):
    predictions.reset_index(drop=True, inplace=True)  # Reset index
    predictions.to_csv(f'{target_name}_predictions.csv', index=False)

# Load the data
data = pd.read_csv('CDOM1.csv')  # Change the filename to your actual CSV file

# Impute NaN values for the entire dataset using KNN
imputer = KNNImputer(n_neighbors=5)
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# Separate features and target variables
X = data_imputed.iloc[:, :-1]  # Exclude the last column (target variable)
y_chla = data_imputed['CDOM']

# Standardize the features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Define the MDN loss function
num_components = 1
def mdn_loss(y_true, y_pred):
    # Extract mean, log_sigma, and alpha using indexing
    num_params = num_components * 3
    mean = y_pred[:, :num_components]
    log_sigma = y_pred[:, num_components:2*num_components]
    alpha = y_pred[:, 2*num_components:]

    sigma = K.exp(log_sigma)

    # Gaussian probability density function
    pdf = K.exp(-0.5 * K.square((y_true - mean) / (sigma + 1e-8))) / (sigma + 1e-8) / np.sqrt(2 * np.pi)

    # Mixture of Gaussians
    loss = -K.log(K.sum(alpha * pdf, axis=1, keepdims=True) + 1e-8)
    return loss

# Function to fine-tune the existing MDN model
def fine_tune_mdn_model(existing_model, X, y, target_name, learning_rate=0.001):
    # Implement early stopping
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

    # Compile the existing model with the same loss function and optimizer
    existing_model.compile(optimizer=Adam(learning_rate=learning_rate), loss=mdn_loss)

    # Fine-tune the model on the new data
    existing_model.fit(
        X, y,
        epochs=100,  # You can adjust the number of epochs
        batch_size=32,
        callbacks=[early_stopping],
        verbose=1
    )

    # Save the fine-tuned MDN model
    existing_model.save(f'{target_name}_fine_tuned_mdn_model.h5')

    # Print the hyperparameters
    print("Hyperparameters:")
    print(existing_model.get_config()['layers'][1]['config'])  # Assuming the first layer is Dense

    # Generate predictions on the fine-tuned model
    y_pred = existing_model.predict(X)

    # Calculate and print MAPE
    mape = mean_absolute_percentage_error(y, y_pred[:, :1].flatten())  # Use mean for prediction
    print(f'MAPE for {target_name} after fine-tuning: {mape}')

    # Save predictions to CSV
    predictions_df = pd.DataFrame({'Actual': y, 'Predicted': y_pred[:, :1].flatten()})
    save_predictions(predictions_df, target_name)

    return mape

# Load the existing MDN model
existing_mdn_model = load_model('CDOM_mdn_model.h5', custom_objects={'mdn_loss': mdn_loss})

# Fine-tune the MDN model for Chla
mape_chla_fine_tuned = fine_tune_mdn_model(existing_mdn_model, X_scaled, y_chla, 'CDOM')

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: nan
Epoch 2/100
1/1 [==============================] - 0s 12ms/step - loss: nan
Epoch 3/100
1/1 [==============================] - 0s 11ms/step - loss: nan
Epoch 4/100
1/1 [==============================] - 0s 11ms/step - loss: nan
Epoch 5/100
1/1 [==============================] - 0s 13ms/step - loss: nan
Epoch 6/100
1/1 [==============================] - 0s 15ms/step - loss: nan
Epoch 7/100
1/1 [==============================] - 0s 10ms/step - loss: nan
Epoch 8/100
1/1 [==============================] - 0s 15ms/step - loss: nan
Epoch 9/100
1/1 [==============================] - 0s 14ms/step - loss: nan
Epoch 10/100
1/1 [==============================] - 0s 14ms/step - loss: nan
Hyperparameters:
{'name': 'dense_1240', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': (None, 8), 'units': 128, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': Non